In [22]:
import sys, os
from pathlib import Path

sys.path.append(os.path.abspath(Path().resolve().parent/'src'))

In [23]:
from kinematicsrobotics.datahandler import Extract
from kinematicsrobotics.dataprocessing import Preprocessing
from kinematicsrobotics.model import Model
from kinematicsrobotics.metrics import Metrics
from kinematicsrobotics.kinematics import Robo

## Dataset

In [24]:
ext = Extract()
ext._path_project = os.path.abspath(Path().resolve().parent)

dataset = ext.dataframe( r'src\data\ready\dataset-radius-1cm.csv')

In [25]:
data  = Preprocessing(dataset = dataset, 
                           x_labels=['p_x', 'p_y','p_z', 'roll', 'pich', 'yaw'],
                           y_labels=['theta_1', 'theta_2', 'theta_3', 'theta_4']
                        )

In [26]:
x_train, x_test, y_train, y_test = data.data_train_test

## Modelo

In [44]:
mlp = Model.mlp_regressor(hidden_layer_sizes=(300,300,300,300,300), EPOCHS_NOCHANGE = 20, activation = 'tanh')

## Estimação de Parâmatros

In [19]:
# Estimação de hiperparâmetros
history = ext.dataframe(r'src\data\ready\history.csv')

history_best = history[history['rank_test_score'] == 1]

params = eval(history_best.iloc[0]['params'])

## Treino

In [8]:
mlp.set_model(**params)

In [37]:
mlp.fit(x=x_train,y=y_train)

## Metricas

In [38]:
metrica = Metrics(model = mlp, 
                  preprocessing = data)

In [39]:
metrica.mse(mlp.predict(x =x_train), y_train)

array([0.00061889, 0.00226159, 0.00574923, 0.00262829])

### Erro juntas

In [40]:
metrica.predict_joint(x = x_train)

array([[ 6.47682686e+01,  1.20243953e+02, -8.52568631e-03,
         4.85449667e+01],
       [ 2.61599523e+01,  1.22275409e+02,  3.57050462e+00,
         9.63573003e+01],
       [ 1.19385802e+02,  8.32989492e+01,  7.09140524e+01,
         4.90455347e+01],
       ...,
       [ 1.22136824e+02,  1.08918060e+02,  2.57386940e+01,
         1.29119207e+01],
       [ 1.10432848e+02,  8.51579233e+01,  9.55981062e+01,
         3.43069709e+01],
       [ 1.18758481e+02,  8.25216609e+01,  2.14287409e+01,
         1.10083072e+01]])

In [41]:
metrica.mse_joint(x = x_train, y = y_train)

array([0.8155132 , 2.28840407, 6.86808   , 3.70235967])

### Erro cartesiano

In [42]:
Elos = [['theta_1',10,0,90,0],
        ['theta_2',0,18,180,0],
        ['theta_3',0,18,-180,0],
        ['theta_4',0,0,90,90],
        ['theta_5',18,0,0,0]
]

robo = Robo("Robo",Elos)

metrica.robotic(robo = robo)

In [43]:
metrica.mse_operacional(x = x_train)

array([326.54744612, 218.40029333, 192.31173064])